<a href="https://colab.research.google.com/github/Mathias0210/Bach/blob/main/Pyth2fors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install mlxtend

In [ ]:
from google.colab import files
uploaded = files.upload()
df = pd.read_excel('SalesData.xlsx')
# show the first few rows
df.head()

In [6]:
# Prepend '_' to ProduktID
df['ProduktID'] = df['ProduktID'].apply(lambda x: '_'+str(x))

#Sætter alle produktnavne til stort
df['ProductName']=df['ProductName'].str.upper()

In [ ]:
# DataFrame for building the recommendation system
orders = df[['TransaktionsID', 'ProduktID']]
print(orders.shape)
print(orders.head(10))
# DataFrame for retrieving product descriptions
products = df[['ProduktID', 'ProductName']].copy()

print(products.shape)
print(products.head(10))

In [ ]:
# Drop duplicated products
products = products[~products.duplicated()]


print(products.shape)
print(products.head(10))

In [48]:
# Fra tidligere trin er der nu kun ca. 6500 linjer i produkt men det bemærkes 

products.to_excel('Products.xlsx')
files.download('Products.xlsx')

#Det bemærkes at forekllige productnavne har samme produktID, ved nærmere inspektion er det dog de samme produkter bare med forskellige navne.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Fjerner duplikerede linjer med productID og beholder


products = (products
.sort_values(by=['ProduktID'],ascending=False))

products.head(160)

In [ ]:
#Fjerner fejlene fra ProdutData ved at overskrive dubletterne

products = products.drop_duplicates(subset='ProduktID', keep="last")
print(products.shape)
products.head(160)

In [ ]:
#Der blevv fjernet lidt over 100 fejl med denne metode og jeg må antage at produkterne er de samme og tabellen nu er klar til brug


#Jeg går nu videre til min liste af transaktioner
#Et loop der laver en liste med alle produktid'er for hver transaktion

def string_list(x):
    return [str(i) for i in x]

orders = orders.groupby('TransaktionsID')['ProduktID'].apply(list).reset_index()

#Der var sneget sig et par ekstra [] ind i produktname som jeg her fjerner

df['ProduktID'] = df['ProduktID'].str.strip('[]')
orders.head(100)


In [113]:
#orders.to_excel('orders.xlsx')
#files.download('orders.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
#Importerer mlxtend https://towardsdatascience.com/mlxtend-a-python-library-with-interesting-tools-for-data-science-tasks-d54c723f89cd

from mlxtend.preprocessing import TransactionEncoder

In [ ]:
# fit the TransactionEncoder
te = TransactionEncoder()

te.fit(orders['ProduktID'])

# do the transformation
orders_1hot = te.transform(orders['ProduktID'])

#Omdanner til en matrix med produktID som kolonner og transaktioner som rækker.
orders_1hot = pd.DataFrame(orders_1hot, columns =te.columns_)
orders_1hot.head()

In [ ]:
#Importerer apriori som er vorres unsupervised metode
from mlxtend.frequent_patterns import apriori

#Instantierer apriori med datasettet der er klargjort og viser første associationer

ap = apriori(orders_1hot, min_support=0.01, max_len=5, use_colnames=True)
ap.head(6)

In [ ]:
from mlxtend.frequent_patterns import association_rules

#Viser her de stærkeste associationer
rules = association_rules(ap,metric="lift", min_threshold = 0.0001)
print(rules)

In [102]:
#Funktion der laver foreslag baseret på produktID

def predict(antecedent, rules, max_results= 6):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    
    # a way to convert a frozen set with one element to string
    preds = preds['consequents'].apply(iter).apply(next)
    
    return preds[:max_results]
  

    preds = predict({'_1017036338'}, rules)
    preds

    # Display the descriptions of the predictions
    for produktID in preds:  
      print(products[produktID])  


      #Konklusion, der er for mange produkter, for få transaktioner og ikke stærke nok associationer til at Apriori kan give anbefalinger...